## Observations and Insights 

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
#print(mouse_df.head())
study_results = pd.read_csv(study_results_path)
#print(results_df.head())

# Combine the data into a single dataset
combined_df = pd.concat(map(pd.read_csv, ['data/Mouse_metadata.csv', 'data/Study_results.csv']),sort=False)
combined_df.tail()

,Mouse ID,Drug Regimen,Sex,Age_months,Weight (g),Timepoint,Tumor Volume (mm3),Metastatic Sites
1888,r944,NaN,NaN,NaN,NaN,45.0,41.581521,2.0
1889,u364,NaN,NaN,NaN,NaN,45.0,31.023923,3.0
1890,p438,NaN,NaN,NaN,NaN,45.0,61.433892,1.0
1891,x773,NaN,NaN,NaN,NaN,45.0,58.634971,4.0
1892,b879,NaN,NaN,NaN,NaN,45.0,72.555239,2.0


In [42]:
# Checking the number of mice in the DataFrame.
mice_value_count = combined_df['Mouse ID'].value_counts()   #2142 combined DF
mice_value_count

g989    14
d251    11
q633    11
t724    11
f345    11
        ..
h428     2
t573     2
n482     2
f932     2
x336     2
Name: Mouse ID, Length: 249, dtype: int64

In [44]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
mouse_time_df = combined_df.loc[:, ['Mouse ID', 'Timepoint']]
dup_mouse = mouse_time_df.groupby(['Mouse ID'])
dup_mouse.head()
#unique_mice_time = mouse_time_df['Mouse ID'].unique() #249 
#unique_mice_time.head()


,Mouse ID,Timepoint
0,k403,NaN
1,s185,NaN
2,x401,NaN
3,m601,NaN
4,g791,NaN
...,...,...
913,v923,15.0
914,w575,15.0
915,t198,15.0
916,n304,15.0


In [39]:
# Optional: Get all the data for the duplicate mouse ID. 


In [17]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
cleaned_df = combined_df['Mouse ID'].unique()
cleaned_df


array(['k403', 's185', 'x401', 'm601', 'g791', 's508', 'f966', 'm546',
       'z578', 'j913', 'u364', 'n364', 'y793', 'r554', 'm957', 'c758',
       't565', 'a644', 'i177', 'j989', 'i738', 'a520', 'w914', 'r811',
       'g288', 'i334', 'q610', 'd251', 'l897', 'c458', 'b742', 'b128',
       'j246', 'a411', 'j119', 'w150', 'v923', 'g316', 's710', 'l509',
       'r944', 'e662', 'u196', 'q597', 'a444', 'i557', 'r921', 'w678',
       'y449', 'a203', 'a251', 'a262', 'a275', 'a366', 'a401', 'a457',
       'a492', 'a577', 'a685', 'a699', 'a788', 'a818', 'a897', 'a963',
       'b313', 'b447', 'b487', 'b559', 'b759', 'b879', 'c139', 'c264',
       'c282', 'c302', 'c326', 'c402', 'c559', 'c580', 'c757', 'c766',
       'c819', 'c832', 'c895', 'c927', 'd133', 'd164', 'd474', 'e213',
       'e227', 'e291', 'e476', 'e584', 'f129', 'f234', 'f278', 'f345',
       'f394', 'f436', 'f545', 'f932', 'f993', 'g107', 'g296', 'g497',
       'g558', 'g570', 'g867', 'g989', 'h246', 'h333', 'h428', 'h531',
      

In [7]:
# Checking the number of mice in the clean DataFrame.
cleaned_mouse_count = cleaned_df['mouse']


## Summary Statistics

In [7]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# This method is the most straighforward, creating multiple series and putting them all together at the end.

In [8]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

## Bar Plots

In [9]:
# Generate a bar plot showing the number of mice per time point for each treatment throughout the course of the study using pandas. 

In [10]:
# Generate a bar plot showing the number of mice per time point for each treatment throughout the course of the study using pyplot.

## Pie Plots

In [11]:
# Generate a pie plot showing the distribution of female versus male mice using pandas

In [12]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot

## Quartiles, Outliers and Boxplots

In [13]:
# Calculate the final tumor volume of each mouse across four of the most promising treatment regimens. Calculate the IQR and quantitatively determine if there are any potential outliers. 


In [14]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest

## Line and Scatter Plots

In [15]:
# Generate a line plot of time point versus tumor volume for a mouse treated with Capomulin


In [16]:
# Generate a scatter plot of mouse weight versus average tumor volume for the Capomulin regimen



## Correlation and Regression

In [17]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
